In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms # 将图片转化为张量形式
import torchvision.models as models # 用来加载已经训练好的VGG19

from PIL import Image    # 用来读取图片
from torchvision.utils import save_image # 用来存储突破

### 1： 该模型用到了预先训练好的VGG19模型，并且在任务中该卷积模型的参数是固定的

In [2]:
model =models.vgg19(pretrained=True).features
print(model)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo

从上面可以看到VGG19一共有36层，对content 和 style 来说 只要选择其中的五层即可 我们按论文中写的选择的是[0,5,10,19,28]

In [3]:
# 打印其中的一层可以看到它的结构是如何的
print(model[(28)])

Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


### 下面写一个方法，该方法能够直接得到对应输入的五个输出特征

In [4]:
class VGG(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.chosen_features = [0,5,10,19,28]
        self.model = models.vgg19(pretrained=True).features[:29]  # 该网络的28层后面层去我们一点关系都没有了
        
    
    def forward(self,x):
        
        features = []
        
        for layer_num, layer in enumerate(self.model):
            x = layer(x)
            if layer_num in self.chosen_features:
                features.append(x)
                
        return features

### 2：加载数据集，主要是对content image 和 style image的图片进行预处理

In [5]:
loader = transforms.Compose(
    [
        transforms.Resize((356,356)),
        transforms.ToTensor()
    ]
)

In [6]:
def load_image(image_name):
    image = Image.open(image_name)
    image = loader(image).unsqueeze(0) # 该方法将张量从3维变成4维的，满足网络模型输入参数的输入条件
    
    return image.to(device)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available else "cpu")
print(device)

cuda


In [8]:
content_image = load_image("content_image/face.jpg")
style_image = load_image("style_image/scream.jpg")

In [9]:
print(content_image.shape)
print(style_image.shape)

torch.Size([1, 3, 356, 356])
torch.Size([1, 3, 356, 356])


###  模型初始化

In [10]:
# 超参数的设计
total_steps = 30000
learning_rate = 0.001
alpha = 0.00001
bate = 1

Model = VGG().to(device).eval()
generated = content_image.clone().requires_grad_(True)
optimizer = optim.Adam([generated],lr=learning_rate)

### 模型训练

In [11]:
for step in range(total_steps):
    
    content_loss = 0
    style_loss = 0
    
    generated_features = Model(generated)
    content_features = Model(content_image)
    style_features = Model(style_image)
    
    for gen_feature,content_feature,style_feature in zip(generated_features,content_features,style_features):
        batch_size,channel,height,width = gen_feature.shape
        
        # 先计算content loss
        content_loss += torch.mean((gen_feature-content_feature)**2)
        
        
        # 后计算style loss 用gram矩阵
        G = gen_feature.view(channel,height*width).mm(gen_feature.view(channel,height*width).t())
        S = style_feature.view(channel,height*width).mm(style_feature.view(channel,height*width).t())
        style_loss += torch.mean((G-S)**2)
        
    total_loss = alpha*content_loss+bate*style_loss
    
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    
    if step % 200 == 0:
        print(content_loss,style_loss,total_loss)
        save_image(generated,"face" + f"/gen_{step}.png")

D:\Tools\miniConda\envs\gan\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


tensor(0., device='cuda:0', grad_fn=<AddBackward0>) tensor(19342456., device='cuda:0', grad_fn=<AddBackward0>) tensor(19342456., device='cuda:0', grad_fn=<AddBackward0>)
tensor(8.7208, device='cuda:0', grad_fn=<AddBackward0>) tensor(938070., device='cuda:0', grad_fn=<AddBackward0>) tensor(938070., device='cuda:0', grad_fn=<AddBackward0>)
tensor(9.6860, device='cuda:0', grad_fn=<AddBackward0>) tensor(453392.2188, device='cuda:0', grad_fn=<AddBackward0>) tensor(453392.2188, device='cuda:0', grad_fn=<AddBackward0>)
tensor(10.2112, device='cuda:0', grad_fn=<AddBackward0>) tensor(301595.7188, device='cuda:0', grad_fn=<AddBackward0>) tensor(301595.7188, device='cuda:0', grad_fn=<AddBackward0>)
tensor(10.5512, device='cuda:0', grad_fn=<AddBackward0>) tensor(234105.2188, device='cuda:0', grad_fn=<AddBackward0>) tensor(234105.2188, device='cuda:0', grad_fn=<AddBackward0>)
tensor(10.8184, device='cuda:0', grad_fn=<AddBackward0>) tensor(193711.7969, device='cuda:0', grad_fn=<AddBackward0>) tensor

tensor(13.4470, device='cuda:0', grad_fn=<AddBackward0>) tensor(19983.5547, device='cuda:0', grad_fn=<AddBackward0>) tensor(19983.5547, device='cuda:0', grad_fn=<AddBackward0>)
tensor(13.4737, device='cuda:0', grad_fn=<AddBackward0>) tensor(19663.8086, device='cuda:0', grad_fn=<AddBackward0>) tensor(19663.8086, device='cuda:0', grad_fn=<AddBackward0>)
tensor(13.4724, device='cuda:0', grad_fn=<AddBackward0>) tensor(19368.4082, device='cuda:0', grad_fn=<AddBackward0>) tensor(19368.4082, device='cuda:0', grad_fn=<AddBackward0>)
tensor(13.4779, device='cuda:0', grad_fn=<AddBackward0>) tensor(19147.2695, device='cuda:0', grad_fn=<AddBackward0>) tensor(19147.2695, device='cuda:0', grad_fn=<AddBackward0>)
tensor(13.4920, device='cuda:0', grad_fn=<AddBackward0>) tensor(18856.3555, device='cuda:0', grad_fn=<AddBackward0>) tensor(18856.3555, device='cuda:0', grad_fn=<AddBackward0>)
tensor(13.4980, device='cuda:0', grad_fn=<AddBackward0>) tensor(18660.2480, device='cuda:0', grad_fn=<AddBackward0>

tensor(13.7487, device='cuda:0', grad_fn=<AddBackward0>) tensor(15214.4404, device='cuda:0', grad_fn=<AddBackward0>) tensor(15214.4404, device='cuda:0', grad_fn=<AddBackward0>)
tensor(13.7948, device='cuda:0', grad_fn=<AddBackward0>) tensor(14370.2676, device='cuda:0', grad_fn=<AddBackward0>) tensor(14370.2676, device='cuda:0', grad_fn=<AddBackward0>)
tensor(13.7935, device='cuda:0', grad_fn=<AddBackward0>) tensor(14314.0791, device='cuda:0', grad_fn=<AddBackward0>) tensor(14314.0791, device='cuda:0', grad_fn=<AddBackward0>)
tensor(13.7949, device='cuda:0', grad_fn=<AddBackward0>) tensor(14251.5654, device='cuda:0', grad_fn=<AddBackward0>) tensor(14251.5654, device='cuda:0', grad_fn=<AddBackward0>)
tensor(13.8082, device='cuda:0', grad_fn=<AddBackward0>) tensor(14257.3877, device='cuda:0', grad_fn=<AddBackward0>) tensor(14257.3877, device='cuda:0', grad_fn=<AddBackward0>)
tensor(13.8129, device='cuda:0', grad_fn=<AddBackward0>) tensor(14236.3867, device='cuda:0', grad_fn=<AddBackward0>

tensor(13.9035, device='cuda:0', grad_fn=<AddBackward0>) tensor(13157.7441, device='cuda:0', grad_fn=<AddBackward0>) tensor(13157.7441, device='cuda:0', grad_fn=<AddBackward0>)
tensor(13.9069, device='cuda:0', grad_fn=<AddBackward0>) tensor(13065.6758, device='cuda:0', grad_fn=<AddBackward0>) tensor(13065.6758, device='cuda:0', grad_fn=<AddBackward0>)
tensor(13.9054, device='cuda:0', grad_fn=<AddBackward0>) tensor(13054.1309, device='cuda:0', grad_fn=<AddBackward0>) tensor(13054.1309, device='cuda:0', grad_fn=<AddBackward0>)
tensor(13.9108, device='cuda:0', grad_fn=<AddBackward0>) tensor(13045.5000, device='cuda:0', grad_fn=<AddBackward0>) tensor(13045.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(13.9270, device='cuda:0', grad_fn=<AddBackward0>) tensor(13122.1250, device='cuda:0', grad_fn=<AddBackward0>) tensor(13122.1250, device='cuda:0', grad_fn=<AddBackward0>)
tensor(13.9140, device='cuda:0', grad_fn=<AddBackward0>) tensor(13021.7207, device='cuda:0', grad_fn=<AddBackward0>